In [13]:
from tqdm.auto import tqdm


import torch
from torch import nn
import torch.optim as optim
from torchvision import datasets, models, transforms
from torch.optim import Adam
import time, os
from PIL import Image
from torch.optim import lr_scheduler
from tempfile import TemporaryDirectory


In [ ]:
BATCH_SIZE = 32
EPOCHS = 5
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {DEVICE}")

# --- ImageNet normalization (CRUCIAL for pretrained models) ---
imagenet_mean = [0.485, 0.456, 0.406]
imagenet_std  = [0.229, 0.224, 0.225]

# --- Transforms ---
train_tf = transforms.Compose([
    transforms.Resize(256),
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(imagenet_mean, imagenet_std),
])

eval_tf = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(imagenet_mean, imagenet_std),
])

# --- Datasets ---
data_root = '/Users/dylancanning/Desktop/UIB/AprenentatgeAutomatic/aa_2526/data/tiny-imagenet-200'

train_full = datasets.ImageFolder(root=f'{data_root}/train', transform=train_tf)
test_ds    = datasets.ImageFolder(root=f'{data_root}/test',  transform=eval_tf)

# Split train into train/val (e.g. 80/20)
val_size = int(0.2 * len(train_full))
train_size = len(train_full) - val_size
train_ds, val_ds = random_split(train_full, [train_size, val_size])

# --- DataLoaders ---
num_workers = 4 if torch.cuda.is_available() else 2
pin_memory = torch.cuda.is_available()

train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True,  num_workers=num_workers, pin_memory=pin_memory)
val_loader   = DataLoader(val_ds,   batch_size=BATCH_SIZE, shuffle=False, num_workers=num_workers, pin_memory=pin_memory)
test_loader  = DataLoader(test_ds,  batch_size=BATCH_SIZE, shuffle=False, num_workers=num_workers, pin_memory=pin_memory)

# --- Sanity check ---
print(f"Train: {len(train_ds)} | Val: {len(val_ds)} | Test: {len(test_ds)}")


In [3]:
img, target = next(iter(train_loader))
print(img.shape, target)

torch.Size([4, 3, 64, 64]) tensor([ 73,  57, 174, 169])


In [4]:
alex = models.alexnet(weights=models.AlexNet_Weights.IMAGENET1K_V1)

print("-" * 50)
print("Arquitectura AlexNet")
print("-" * 50)
print(alex)

--------------------------------------------------
Arquitectura AlexNet
--------------------------------------------------
AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classi

In [5]:
alex.features[0]

Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))

In [20]:
model_conv = alex
for param in model_conv.parameters():
    param.requires_grad = False

# Parameters of newly constructed modules have requires_grad=True by default
# num_ftrs = model_conv.fc.in_features
my_model = nn.Sequential(
    alex,
    nn.Linear(1000, 200)
)

my_model = my_model.to(DEVICE)

loss_function = nn.CrossEntropyLoss()

# Observe that only parameters of final layer are being optimized as
# opposed to before.
optimizer = optim.SGD(my_model.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

In [21]:
def get_batch_accuracy(output, y, N):
    zero_tensor = torch.tensor([0]).to(DEVICE)
    pred = torch.gt(output, zero_tensor)
    correct = pred.eq(y.view_as(pred)).sum().item()
    return correct / N

def train(model, check_grad=False):
    total_loss = 0
    total_correct = 0

    model.train()
    for x, y in train_loader:
        x, y = x.to(DEVICE), y.to(DEVICE)
        output = model(x)

        optimizer.zero_grad()
        batch_loss = loss_function(output, y)
        batch_loss.backward()
        optimizer.step()

        total_loss += batch_loss.item()
        preds = output.argmax(dim=1)
        total_correct += (preds == y).sum().item()

    acc = total_correct / len(train_loader.dataset)
    print(f"Train - Loss: {total_loss:.4f} Accuracy: {acc:.4f}")

def validate(model):
    total_loss = 0
    total_correct = 0

    model.eval()
    with torch.no_grad():
        for x, y in test_loader:
            x, y = x.to(DEVICE), y.to(DEVICE)
            output = model(x)
            total_loss += loss_function(output, y).item()
            preds = output.argmax(dim=1)
            total_correct += (preds == y).sum().item()

    acc = total_correct / len(test_loader.dataset)
    print(f"Test - Loss: {total_loss:.4f} Accuracy: {acc:.4f}")


In [22]:
epochs = 10

for epoch in range(epochs):
    print('Epoch: {}'.format(epoch))
    train(my_model, check_grad=False)
    validate(my_model)

Epoch: 0


KeyboardInterrupt: 

In [ ]:
self.running_mean = (1-self.momentum)*self.running_mean + self.momentum*batch_mean
self.running_var = (1-self.momentum)*self.running_var + self.momentum*batch_var
x_hat = (x-batch_mean) / torch.sqrt(batch_var + self.eps)